<a href="https://colab.research.google.com/github/rrbaichw/CSC422-NBA-Analyzer/blob/main/2018_NBA_Analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

import pandas as pd
import matplotlib.pyplot as plt
import sklearn.metrics
from sklearn import datasets


In [ ]:
import pandas as pd



# Dictionary mapping positions to their respective weight mappings
position_weights = {
    'G': {'PPG': 0.3, 'FG': 0.3, 'APG': 0.25, 'FTA': 0.15, 'RPG': 0.1},
    'SG': {'PPG': 0.3, 'FG': 0.3, 'FTA': 0.15, 'APG': 0.15, 'RPG': 0.1},
    'SF': {'PPG': 0.3, 'FG': 0.3, 'RPG': 0.15, 'APG': 0.15, 'FTA': 0.1},
    'PF': {'PPG': 0.3, 'RPG': 0.3, 'BPG': 0.15, 'APG': 0.15, 'SPG': 0.1},
    'C': {'RPG': 0.3, 'BPG': 0.3, 'PPG': 0.2, 'SPG': 0.1, 'FTA': 0.1}
}

def calculate_weighted_rating(player_row):
    position = player_row['POS']
    # Check if the position is directly in the position_weights dictionary
    if position in position_weights:
        weights = position_weights[position]
    elif position in ['F', 'SF', 'PF']:
        weights = position_weights['SF']
    elif position in ['G', 'SG', 'PG']:
        weights = position_weights['G']
    elif position == 'G-F' or position == 'F-G':
        # Average the weights for 'G' and 'SF'
        weights = {stat: (position_weights['G'].get(stat, 0) + position_weights['SF'].get(stat, 0)) / 2
                   for stat in set(position_weights['G']) | set(position_weights['SF'])}
    elif position == 'C-F' or position == 'F-C':
        # Example: Average the weights for 'C' and 'SF'
        weights = {stat: (position_weights['C'].get(stat, 0) + position_weights['SF'].get(stat, 0)) / 2
                   for stat in set(position_weights['C']) | set(position_weights['SF'])}
    else:
        print(f"Unhandled position: {position}")
        return 0
    # Calculate the weighted rating
    rating = sum(player_row.get(stat, 0) * weight for stat, weight in weights.items())

    if player_row.name < 5:
        print(f"Calculating rating for {player_row['NAME']}, POS: {position}")
        for stat, weight in weights.items():
            print(f"Stat: {stat}, Weight: {weight}, Value: {player_row.get(stat, 0)}")
        print(f"Rating: {rating}\n")

    return rating



def load_data(file_path):
    df = pd.read_csv(file_path)
    if 'Rating' not in df.columns:
        df['Rating'] = df.apply(calculate_weighted_rating, axis=1)
    return df

# Calculate ratings for each player in the dataset
def calculate_player_ratings(dataframe):
    dataframe['Rating'] = dataframe.apply(calculate_weighted_rating, axis=1)
    return dataframe[['NAME', 'TEAM', 'POS', 'Rating']]

# Main function to run the program
def main():
    file_path = 'nba_stats_2017.csv'
    player_stats = load_data(file_path)
    player_rankings = calculate_player_ratings(player_stats)


    print(player_rankings)

if __name__ == '__main__':
    main()


In [ ]:
import pandas as pd


def load_data(file_path):
    return pd.read_csv(file_path)

def display_teams(player_stats):
    teams = player_stats['TEAM'].unique()
    print("NBA Teams:")
    for team in sorted(teams):
        print(team)
    print("\n")

def get_team_selection(teams):
    team1 = input("Select the first team: ")
    while team1 not in teams:
        print("Team not recognized. Please try again.")
        team1 = input("Select the first team: ")

    team2 = input("Select the second team: ")
    while team2 not in teams or team2 == team1:
        print("Team not recognized or same as the first team. Please try again.")
        team2 = input("Select the second team: ")

    return team1, team2


def calculate_team_strength(player_stats, team_stats, team, top_n_players=7) :
    # Select top N players based on rating
    team_players = player_stats[player_stats['TEAM'] == team].sort_values(by='Rating', ascending=False).head(top_n_players)

    weights = [1 - 0.2 * i for i in range(top_n_players)]
    team_strength_from_players = sum(rating * weight for rating, weight in zip(team_players['Rating'], weights))

    if team in team_stats['TEAM'].values:
        # Fetch team win-loss ratio safely and apply as a weightage
        team_record = team_stats[team_stats['TEAM'] == team]
        if not team_record.empty:
            # calculation of win percentage
            team_win_loss_ratio = team_record['W'].iloc[0] / (team_record['W'].iloc[0] + team_record['L'].iloc[0])

            # Combine player strength with team performance
            team_strength = team_strength_from_players * (1 + team_win_loss_ratio)
        else:
            team_strength = team_strength_from_players
    else:
        print(f"Team '{team}' not found in team statistics.")
        team_strength = team_strength_from_players

    return team_strength

def predict_matchup(player_stats, team_stats, team1, team2):
    team1_strength = calculate_team_strength(player_stats, team_stats, team1, top_n_players=7)
    team2_strength = calculate_team_strength(player_stats, team_stats, team2, top_n_players=7)
    total_strength = team1_strength + team2_strength
    team1_probability = round((team1_strength / total_strength) * 100)
    team2_probability = 100 - team1_probability

    print(f"\n{team1} Total Strength: {team1_strength}")
    print(f"{team2} Total Strength: {team2_strength}")
    print(f"\n{team1} has a {team1_probability}% chance of winning.")
    print(f"{team2} has a {team2_probability}% chance of winning.")

    if team1_strength > team2_strength:
        print(f"\nPredicted Winner: {team1}")
    elif team2_strength > team1_strength:
        print(f"\nPredicted Winner: {team2}")
    else:
        print("\nIt's a tie!")

def main():
    player_file_path = 'nba_stats_2017.csv'
    team_stats_file_path = 'team_stats2017.csv'

    # Load player and team stats
    player_stats = load_data(player_file_path)
    team_stats = pd.read_csv(team_stats_file_path)

    # Standardize team names in both DataFrames
    team_stats['TEAM'] = team_stats['TEAM'].str.upper().str.strip()
    player_stats['TEAM'] = player_stats['TEAM'].str.upper().str.strip()

    # Calculate player ratings after loading the data
    player_stats = calculate_player_ratings(player_stats)

    # Calculate win-loss ratio for team stats
    team_stats['WIN%'] = team_stats['W'] / (team_stats['L'] + 1)  # Prevent division by zero

    # Display available teams
    display_teams(player_stats)

    # User selects two teams to match up
    teams = player_stats['TEAM'].unique()
    team1, team2 = get_team_selection(teams)

    # Predict the outcome of the matchup
    predict_matchup(player_stats, team_stats, team1, team2)

if __name__ == '__main__':
    main()
